<a href="https://colab.research.google.com/github/skywalker00001/Conterfactual-Reasoning-Project/blob/main/huggingface_t5_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version 6.0

ground truth using "denosing"

find out the different pairs and only output those different things

# 1. Preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = 'drive/MyDrive/LM/'

Mounted at /content/drive


In [2]:
!pip install sentencepiece
!pip install transformers -q
!pip install wandb -q

     |████████████████████████████████| 1.2 MB 3.2 MB/s 
     |████████████████████████████████| 3.8 MB 2.1 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 6.5 MB 50.3 MB/s 
     |████████████████████████████████| 1.7 MB 3.2 MB/s 
     |████████████████████████████████| 181 kB 58.4 MB/s 
     |████████████████████████████████| 144 kB 36.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [3]:
# Importing stock libraries
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import os
import regex as re
import sys
sys.path.append('/content/drive/MyDrive/LM/')
from global_param import MyConfig
import nltk
nltk.download("punkt")
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

import torch
from torch import cuda
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Login to wandb to log the model run and all the parameters
# 7229adacb32965027d73056a6927efd0365a00bc
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
myconfig = MyConfig()

In [6]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Thu Mar 10 20:16:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# # Setting up the device for GPU usage

device = 'cuda' if cuda.is_available() else 'cpu'
print("Device is: ", device)

# Set random seeds and deterministic pytorch for reproducibility
#SEED = 42
SEED = myconfig.SEED
torch.manual_seed(SEED) # pytorch random seed
np.random.seed(SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

Device is:  cuda


In [8]:
# Global Parameter
model_version = "6.3"

load_version = "6.2"
initial_epoch = 0

# WandB – Initialize a new run
wandb.init(project="counterfactual"+model_version)

# WandB – Config is a variable that holds and saves hyperparameters and inputs
# Defining some key variables that will be used later on in the training  
# config = wandb.config          # Initialize config
# config.TRAIN_BATCH_SIZE = 16    # input batch size for training (default: 64)
# config.VALID_BATCH_SIZE = 32    # input batch size for testing (default: 1000)
# config.TRAIN_EPOCHS = 51        # number of epochs to train (default: 10)
# config.VAL_EPOCHS = 1 
# config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
# config.SEED = 42               # random seed (default: 42)
# config.SOURCE_LEN = 150
# config.TARGET_LEN = 110

# WandB – Config is a variable that holds and saves hyperparameters and inputs
# Defining some key variables that will be used later on in the training  
config = wandb.config          # Initialize config
config.TRAIN_BATCH_SIZE = 16    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 32    # input batch size for testing (default: 1000)
#config.TRAIN_EPOCHS = myconfig.TRAIN_EPOCHS        # number of epochs to train (default: 10)
config.TRAIN_EPOCHS = 41
config.VAL_EPOCHS = myconfig.VAL_EPOCHS 
config.LEARNING_RATE = myconfig.LEARNING_RATE    # learning rate (default: 0.01)
config.SEED = myconfig.SEED              # random seed (default: 42)
config.SOURCE_LEN = 150
config.TARGET_LEN = 70
config.LOAD_PATH = root+'models/model'+load_version+'.tar'
config.SAVE_PATH = root+'models/model'+model_version+'.tar'

wandb: Currently logged in as: skywalk3r (use `wandb login --relogin` to force relogin)


In [ ]:
PRETRAINED_MODEL_NAME =  myconfig.PRETRAINED_MODEL_NAME
# tokenzier for encoding the text
t5_tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL_NAME)
model = model.to(device)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

# 2. Load dataframe

In [ ]:
#training df
small_path = root + '/TimeTravel/cleaned_small_2.0.xlsx'
small_df = pd.read_excel(small_path)
#small_df.head()
print(len(small_df))

In [ ]:
small_df.head(3)

In [ ]:
#valid df
large_path = root + '/TimeTravel/cleaned_large_2.0.xlsx'
large_df = pd.read_excel(large_path)
#large_df.head()
print(len(large_df))

In [ ]:
small_ids = []
for i in range(len(small_df)):
    small_ids.append(small_df.loc[i, 'story_id'])

print(len(small_ids))

In [ ]:
large_df = large_df[~large_df.story_id.isin(small_ids)]
large_df = large_df.reset_index(drop=True)   # must reset index after delete rows
print(len(large_df))

In [ ]:
# select data not in training set
part_large_cleaned_df = large_df[0:100]
#part_large_cleaned_df = large_cleaned_df[0:1000]
part_large_cleaned_df = part_large_cleaned_df.reset_index(drop=True)
print(len(part_large_cleaned_df))

# 3. Dataset and Dataloader

In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, input_len, output_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.input_len = input_len
        self.output_len = output_len
        self.input = self.data.input1
        self.output = self.data.output1
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input = str(self.input[index])
        # input = ' '.join(input.split())

        output = str(self.output[index])
        # output = ' '.join(output.split())

        source = self.tokenizer.encode_plus(input, max_length= self.input_len, padding='max_length', return_tensors='pt')
        target = self.tokenizer.encode_plus(output, max_length= self.output_len, padding='max_length', return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
train_df = small_df
valid_df = part_large_cleaned_df

trainingset = CustomDataset(dataframe=train_df, tokenizer=t5_tokenizer, input_len=config.SOURCE_LEN , output_len=config.TARGET_LEN )
validset = CustomDataset(dataframe=valid_df, tokenizer=t5_tokenizer, input_len=config.SOURCE_LEN , output_len=config.TARGET_LEN )

In [ ]:
# max_sou_len = 0
# max_tar_len = 0
# for i in range(len(small_df)):
#     input = small_df.loc[i, 'input1']
#     output = small_df.loc[i, 'output1']
#     source = t5_tokenizer.encode_plus(input, return_tensors='pt')['input_ids'].squeeze()
#     target = t5_tokenizer.encode_plus(output, return_tensors='pt')['input_ids'].squeeze()
#     max_sou_len = max(max_sou_len, len(source))
#     max_tar_len = max(max_tar_len, len(target))


# print(max_sou_len)
# print(max_tar_len)

In [ ]:
# max_sou_len = 0
# max_tar_len = 0
# for i in range(len(large_df)):
#     input = large_df.loc[i, 'input1']
#     output = large_df.loc[i, 'output1']
#     source = t5_tokenizer.encode_plus(input, return_tensors='pt')['input_ids'].squeeze()
#     target = t5_tokenizer.encode_plus(output, return_tensors='pt')['input_ids'].squeeze()
#     max_sou_len = max(max_sou_len, len(source))
#     max_tar_len = max(max_tar_len, len(target))


# print(max_sou_len)
# print(max_tar_len)

In [ ]:
# pick up a data sample
sample_idx = 4
sample = trainingset[sample_idx]

source_ids = sample["source_ids"]
source_mask = sample["source_mask"]
target_ids = sample["target_ids"]
target_ids_y = sample["target_ids_y"]

print(source_ids)

In [ ]:
print(train_df.loc[sample_idx, 'output1'])

sen = t5_tokenizer.decode(target_ids, skip_special_tokens=False) # skip_special_tokens=True will be completely same.
print(sen)
sen = t5_tokenizer.decode(source_ids, skip_special_tokens=False) # skip_special_tokens=True will be completely same.
print(sen)


In [ ]:
# DataLoader

train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 2
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 2
    }

training_loader = DataLoader(trainingset, **train_params)
val_loader = DataLoader(validset, **val_params)
print(len(training_loader))
print(len(val_loader))

# 4. Define train() and val()

In [ ]:
def save_model(epoch, model, optimizer, loss, PATH):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, PATH)

In [ ]:
def load_model(PATH):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, epoch, loss

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for i,data in enumerate(loader):
        #len(loader)=10xx
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)
        y = data['target_ids'].to(device, dtype = torch.long)

        # padded ids (pad=0) are set to -100, which means ignore for loss calculation
        y[y[: ,:] == tokenizer.pad_token_id ] = -100
        label_ids = y.to(device)

        outputs = model(input_ids = ids, attention_mask = mask, labels=label_ids)
        loss = outputs[0]
        #logit = outputs[1]
        
        if i%50 == 0:
            wandb.log({"Training Loss": loss.item()})

        if i%600==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # xm.optimizer_step(optimizer)
        # xm.mark_step()
   
    if (epoch % 5 == 0):
        save_model(epoch, model, optimizer, loss.item(), config.SAVE_PATH)
 

In [ ]:
def validate(tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    raws = []
    final_loss = 0
    with torch.no_grad():
        for i, data in enumerate(loader):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            '''
            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                num_beams=2,
                max_length=config.TARGET_LEN, 
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            '''
            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                num_beams=2,
                max_length=config.TARGET_LEN, 
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            loss = model(input_ids=ids, attention_mask=mask, labels=y).loss
            final_loss += loss

            raw = [tokenizer.decode(i, skip_special_tokens=False) for i in ids]
            preds = [tokenizer.decode(i, skip_special_tokens=False) for i in generated_ids]
            target = [tokenizer.decode(i, skip_special_tokens=False)for i in y]
            if i%3==0:
                print(f'valid Completed {(i+1)* config.VALID_BATCH_SIZE}')

            raws.extend(raw)
            predictions.extend(preds)
            actuals.extend(target)
    return raws, predictions, actuals, final_loss

# 5. main()

In [ ]:
import time
# Helper function to print time between epochs
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# if need, load model

loss = 0
if (load_version != None and load_version != ""):
    model, optimizer, initial_epoch, loss = load_model(config.LOAD_PATH)
print(loss)

In [ ]:
# Log metrics with wandb
#wandb.watch(model, log="all")

# Training loop
print('Initiating Fine-Tuning for the model on counterfactual dataset:')

for epoch in range(initial_epoch, initial_epoch+config.TRAIN_EPOCHS):
#for epoch in tqdm(range(config.TRAIN_EPOCHS)):
    start_time = time.time()
    train(epoch, t5_tokenizer, model, device, training_loader, optimizer)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')

# Mark the run as finished
wandb.finish()

In [ ]:
# Load model
# model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL_NAME)
# model = model.to(device)
# optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

# model, optimizer, epoch, loss = load_model(config.LOAD_PATH)

# 6. Inference

In [ ]:
# # load model
# model, optimizer, initial_epoch, loss = load_model(config.LOAD_PATH)
# print(loss)

In [ ]:
# Validation loop and saving the resulting file with predictions and acutals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now inferecing:')
start_time = time.time()
raws, predictions, actuals,final_loss = validate(t5_tokenizer, model, device, val_loader)
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f'Time: {epoch_mins}m {epoch_secs}s')

final_df = pd.DataFrame({'input_text': raws, 'ground_truth': actuals, 'generated_text': predictions})
#final_df.to_csv(root + 'results/' + 'output' + model_version + '.csv')
final_df.to_excel(root + 'results/' + 'output' + model_version + '.xlsx')
print('Output Files generated for review')
print(f'Final Loss is: {final_loss:.5f}')

In [ ]:
print(len(actuals))

# 7. check the samples with same original ending and edited ending

In [ ]:
# import pandas as pd
# import regex as re

result_df = pd.read_excel(root + 'results/' + 'output_beam1' + model_version + '.xlsx')
result_df.head()
print(len(result_df))

1000


In [ ]:
or_pat = re.compile(r'(original_ending: )(.*)$') 
ed_pat = re.compile(r'(edited_ending: )(.*)$') 
pipei = re.search(ed_pat, result_df.iloc[0].generated_text)
# pipei = re.search(or_pat, result_df.iloc[0].raw_text)
print(pipei.group(2))

Soon, they wanted to spend time with her. She reflected on why she was so nice. She decided she needed to be a more loving person.


In [ ]:
re_pat = re.compile(r'(original_ending: )(.*)$')  # regular expression, pick the  text after "original_ending: "
#orig =  = re.search(re_pat, te).group(2)
or_text = [] # or for original_ending
ed_text = [] # ed for edited_ending

for i in range(len(result_df)):
    or_text.append(re.search(or_pat, result_df.loc[i, "raw_text"]).group(2))
    ed_text.append(re.search(ed_pat, result_df.loc[i, "generated_text"]).group(2))
print(len(or_text))
print(len(ed_text))

1000
1000


In [ ]:
comparison = [i==j for i, j in zip(or_text, ed_text)]
print(comparison)

[False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, True, True, False, True, False, True, False, False, False, True, False, False, True, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, True, False, True, True, True, False, False, False, True, True, False, False, True, True, False, True, False, False, True, True, True, False, True, True, False, False, True, False, False, False, False, False, False, True, False, True, True, False, False, True, False, False, False, False, False, False, False, True, False, True, False, True, False, False, False, False, False, False, True, False, False, True, True, False, False, False, False, False, False, False, False, True, False, True, True, True, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, F

In [ ]:
count =  pd.value_counts(comparison)
print(count)

False    678
True     322
dtype: int64


In [ ]:
result_df[comparison].head(10)

,Unnamed: 0,raw_text,ground_truth,generated_text
5,5,premise: Pam is married to her Husband Jeff. i...,edited_ending: Jeff then became distant and wa...,edited_ending: Jeff isn't the same person anym...
9,9,premise: I am supposed to take food to a party...,edited_ending: I will have to spend all day co...,edited_ending: I spent all day yesterday cooki...
19,19,premise: Eli was a rambunctious child. initial...,edited_ending: Eli's mother constantly tried t...,edited_ending: Eli's mother constantly tried t...
20,20,premise: Jane was on her way to work. initial:...,edited_ending: It was a normal day of work. Ja...,edited_ending: She called her roommate who ver...
22,22,premise: Madeline was doing some gardening. in...,edited_ending: A snake had struck her! Her han...,edited_ending: A bee had stung her! Her hand s...
24,24,premise: Jenny was driving to work. initial: S...,"edited_ending: A car tried to park near hers, ...",edited_ending: A car on her right did not stop...
28,28,premise: Alex knew the solution to a problem. ...,edited_ending: People thought Alex was dumb. A...,edited_ending: The people thought Alex was sma...
31,31,premise: Tiffany was excited to go to college ...,edited_ending: The two of them became very clo...,edited_ending: The three of them became very c...
36,36,premise: Bob wanted to see how many stories he...,edited_ending: Bob couldn't believe he'd told ...,edited_ending: Instead of information he got a...
37,37,premise: Melissa rode her bike to school every...,edited_ending: She was carried away on a stret...,edited_ending: She was carried away on a stret...


In [ ]:
same_df = result_df[comparison]
same_df.reset_index(drop=True)

In [ ]:
same_df.to_excel(root + 'results/' + 'output_same_b1' + model_version + '.xlsx')

## pick some samples to test


In [ ]:
model.eval()
with torch.no_grad():
    text = 'premise: I am supposed to take food to a party tomorrow. initial: I had bought all the ingredients for it last week. counterfactual: I need to buy all the ingredients for it after work today. original_ending: I spent all day yesterday cooking the food. Unfortunately, I burnt the food. I won\'t be able to get new ingredients in time for tomorrow\'s party.'

    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_length=config.TARGET_LEN, num_beams=2)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

edited_ending: I spent all day yesterday cooking the food. Unfortunately, I burnt the food. I won't be able to get new ingredients in time for tomorrow's party.


# 8. Evalutation


## 7.1 Blue score

In [ ]:
# predicitions: y',  actuals: y
from torchtext.data.metrics import bleu_score

In [ ]:
pre_corpus = [i.split(" ") for i in predictions]
act_corpus = [i.split(" ") for i in actuals]

In [ ]:
print(act_corpus)

[['edited_ending:', 'Soon,everyone', 'wanted', 'to', 'spend', 'time', 'with', 'her.', 'She', 'reflected', 'on', 'if', 'she', 'could', 'be', 'nicer.', 'She', 'decided', 'she', 'needed', 'to', 'atleast', 'try', 'to', 'be', 'the', 'best', 'person', 'she', 'could', 'be.'], ['edited_ending:', 'Bill', 'congratulated', 'the', 'other', 'team.', 'He', 'also', 'went', 'into', 'the', 'locker', 'room', 'to', 'cry.', 'He', "didn't", 'tell', 'anyone', 'until', 'the', 'next', 'day.'], ['edited_ending:', 'She', 'jumped', 'over', 'the', 'fence', 'and', 'vanished.', 'Without', 'a', 'spare,', 'the', 'game', 'was', 'done.', 'They', 'were', 'forced', 'to', 'quit.'], ['edited_ending:', 'She', 'was', 'nervous', 'about', 'helping', 'the', 'people', 'and', 'making', 'them', 'feel', 'more', 'comfortable.', 'One', 'day', 'she', 'witnessed', 'something', 'that', 'she', 'could', 'never', 'forget.', 'She', 'decided', 'she', 'could', 'not', 'live', 'with', 'the', 'memories', 'and', 'quit', 'her', 'job.'], ['edited_e

In [ ]:
print(pre_corpus)

[['edited_ending:', 'Soon,', 'they', 'wanted', 'to', 'spend', 'time', 'with', 'her.', 'She', 'reflected', 'on', 'why', 'she', 'was', 'so', 'kind.', 'She', 'decided', 'she', 'needed', 'to', 'be', 'a', 'nicer', 'person.'], ['edited_ending:', 'Bill', 'congratulated', 'the', 'opposing', 'team.', 'He', 'also', 'went', 'into', 'the', 'locker', 'room', 'to', 'cry.', 'He', "didn't", 'tell', 'anyone', 'until', 'the', 'next', 'day.'], ['edited_ending:', 'Then', 'she', 'flew', 'over', 'the', 'fence', 'and', 'vanished.', 'Without', 'a', 'spare,', 'the', 'game', 'was', 'done.', 'They', 'were', 'forced', 'to', 'quit.'], ['edited_ending:', 'She', 'hated', 'helping', 'the', 'people', 'and', 'making', 'them', 'feel', 'more', 'comfortable.', 'One', 'day', 'she', 'witnessed', 'something', 'that', 'she', 'could', 'never', 'forget.', 'She', 'decided', 'she', 'could', 'live', 'with', 'the', 'memories', 'and', 'quit', 'her', 'job.'], ['edited_ending:', 'I', 'tried', 'to', 'stay', 'quiet', 'and', 'not', 'star

In [ ]:
#bs = bleu_score([pre_corpus[0]], [act_corpus[0]], max_n=1, weights=[1])
#bs = bleu_score([pre_corpus[0]], [act_corpus[0]], max_n=2, weights=[0.5, 0.5])
bs_1 = bleu_score(pre_corpus, act_corpus, max_n=1, weights=[1])
#bs_2 = bleu_score(pre_corpus, act_corpus, max_n=2, weights=[0.1, 0.9])
#print(f'bs_1: {bs_1:.5f}, bs_2: {bs_2:.5f}')
print(f'bleus_1: {bs_1:.5f}')

bleus_1: 0.02605


## 7.2 ROUGE

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge 
def compute_rouge(predictions, targets):
    predictions = [" ".join(prediction).lower() for prediction in predictions]
    predictions = [prediction if prediction else "EMPTY" for prediction in predictions]
    targets = [" ".join(target).lower() for target in targets]
    targets = [target if target else "EMPTY" for target in targets]
    rouge = Rouge()
    scores = rouge.get_scores(hyps=predictions, refs=targets, avg=True)
    return scores['rouge-1']['f']

In [ ]:
rouge_1 = compute_rouge(predictions, actuals)
print(f'rouge_1: {rouge_1:.5f}')

rouge_1: 0.96353


## 7.3 T5 loss (cross entropy), discussed before

In [ ]:
print(final_loss / len(part_large_cleaned_df))

tensor(0.9354, device='cuda:0')


In [ ]:
# source = tokenizer.encode_plus(predictions, max_length= config.SOURCE_LEN, padding='max_length', return_tensors='pt')
# target = tokenizer.encode_plus(actuals, max_length= config.TARGET_LEN, padding='max_length', return_tensors='pt')

# source_ids = source['input_ids'].squeeze()
# target_ids = target['input_ids'].squeeze()

In [ ]:
# encode the sources OOM
'''
source_encoding = tokenizer(
    predictions, max_length= config.SOURCE_LEN, padding='max_length', return_tensors='pt'
    )
original, attention_mask = source_encoding.input_ids, source_encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    actuals, max_length= config.TARGET_LEN, padding='max_length', return_tensors='pt'
    )
ending = target_encoding.input_ids
ending[ending == tokenizer.pad_token_id] = -100

original, attention_mask, ending = original.to(device), attention_mask.to(device), ending.to(device)
# forward pass
loss = model(input_ids=original, attention_mask=attention_mask, labels=ending).loss
'''

"\nsource_encoding = tokenizer(\n    predictions, max_length= config.SOURCE_LEN, padding='max_length', return_tensors='pt'\n    )\noriginal, attention_mask = source_encoding.input_ids, source_encoding.attention_mask\n\n# encode the targets\ntarget_encoding = tokenizer(\n    actuals, max_length= config.TARGET_LEN, padding='max_length', return_tensors='pt'\n    )\nending = target_encoding.input_ids\nending[ending == tokenizer.pad_token_id] = -100\n\noriginal, attention_mask, ending = original.to(device), attention_mask.to(device), ending.to(device)\n# forward pass\nloss = model(input_ids=original, attention_mask=attention_mask, labels=ending).loss\n"

In [ ]:
print(a)

I paid the cashier and patiently waited for my drink.


In [ ]:
print(a.splitlines())

['I paid the cashier and patiently waited for my drink.']


In [ ]:
diff = d.compare(a.splitlines(), b.splitlines())

In [ ]:
a = "I paid the cashier and patiently waited for my drink."
b = "I paid the cashier and patiently waited at the counter for my drink."

from difflib import Differ
d = Differ()
diff = d.compare(a.splitlines(), b.splitlines())
print('\n'.join(list(diff)))

- I paid the cashier and patiently waited for my drink.
+ I paid the cashier and patiently waited at the counter for my drink.
?                                         +++++++++++++++



In [ ]:
import difflib
a = "I paid the cashier and patiently waited for my drink."
b = "I paid the cashier and patiently waited at the counter for my drink."
s = difflib.SequenceMatcher(None, a, b)
for block in s.get_matching_blocks():
    print(block)
    


Match(a=0, b=0, size=40)
Match(a=40, b=55, size=13)
Match(a=53, b=68, size=0)


In [ ]:
import difflib
a = "I paid the cashier and patiently waited acoorinding to for my drink."
b = "I paid the cashier and patiently waited at the counter for my drink."
s = difflib.SequenceMatcher(None, a, b)

matches = []
for block in s.get_matching_blocks():
    #matches.append([block[0], block[1], block[2]])
    matches.append([i for i in block])
    #matches.append(block)
print(matches)   


# explanation: matches[i, 0] are the a index, matches[i, 1] are the b index, matches[i, 2] are the lengths of same (matched) words.

[[0, 0, 41], [41, 47, 2], [44, 53, 1], [54, 54, 14], [68, 68, 0]]


In [ ]:
changes = []
for i in range(len(matches) - 1):
    print(matches[i])
    if ((a[i,0]+ a[i,2] < a[i+1,0]) && (b[i,0]+ b[i,2] < b[i+1,0])):     # replacing
        changes.append(f"{a[i,0]+a[i,1]}-{a[i,2]}: {}")

[0, 0, 41]
[41, 47, 2]
[44, 53, 1]
[54, 54, 14]


In [ ]:
print(a)

In [ ]:
print(len(a))

53


In [ ]:
a1 = tokenizer(a)
print(a1)

{'input_ids': [27, 1866, 8, 1723, 972, 11, 1868, 120, 3, 13106, 3, 9, 509, 32, 13119, 53, 12, 21, 82, 3281, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
